In [ ]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.optimizers.legacy import Adam

In [ ]:
from conf.settings import Settings
from estimator.estimatorUtils import callbacks,plot_latent_space
from estimator.VAE import VariationalAutoEncoder
from utils.utils import show_pictures,create_folder,process_data,show_reconstructed

In [ ]:
settings = Settings()

In [ ]:
img_height = settings.img_height
img_width = settings.img_width

batch_size = settings.batch_size
epochs = settings.epochs
learning_rate = settings.learning_rate

metrics_folder = settings.metrics_folder
model_folder = settings.model_folder

rmse_multiplier = settings.rmse_multiplier
latent_space = settings.latent_space

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
show_pictures(x_train,y_train)

In [ ]:
show_pictures(x_test, y_test)

In [ ]:
create_folder(metrics_folder)
create_folder(model_folder)

In [ ]:
x_train, y_train,x_test, y_test = process_data(x_train, y_train,x_test, y_test)

In [ ]:
callbacks_list = callbacks()

In [ ]:
estimator = VariationalAutoEncoder()

In [ ]:
estimator.custom_compile(learning_rate = learning_rate, rmse_multiplier = rmse_multiplier)

In [ ]:
history = estimator.full_model.fit(x_train,x_train,validation_data = (x_test,x_test), epochs = epochs,batch_size = batch_size, callbacks = callbacks_list)

In [ ]:
fig, axs = plt.subplots(2,2)
axs[0, 0].plot(history.history["loss"])
axs[0, 0].plot(history.history["val_loss"])
axs[0, 0].set_title("Total Loss")

axs[0, 1].plot(history.history["vae_r_loss"])
axs[0, 1].plot(history.history["val_vae_r_loss"])
axs[0, 1].set_title("RMSE Loss")

axs[1, 1].plot(history.history["vae_kl_loss"])
axs[1, 1].plot(history.history["val_vae_kl_loss"])
axs[1, 1].set_title("KL Divergence Loss")

axs[1, 0].plot(history.history["lr"])
axs[1, 0].set_title("Learning Rate")
fig.tight_layout()
plt.savefig(metrics_folder + '/losses.png')

In [ ]:
show_reconstructed(x_test,estimator)

In [ ]:
show_reconstructed(x_test,estimator)

In [ ]:
plot_latent_space(x_test,y_test,estimator)